# GitHub Data Collection

In this notebook, we will fetch the GitHub Issue/PR data for repositories we are interested in using the [MI tool](https://github.com/thoth-station/mi), pre-process the raw data into suitable data frames and store them as parquet files to an s3 bucket. We will also create [Trino](https://trino.io/) tables for the generated parquet files so that we can later create dashboards in [Superset](https://superset.operate-first.cloud/).

In [1]:
import os
from dotenv import find_dotenv, load_dotenv
import warnings
import trino
from s3_communication import S3Communication

warnings.filterwarnings("ignore")
load_dotenv(find_dotenv())

True

In [2]:
## Create a .env file on your local with the correct configs
REPO = os.getenv("REPO")
ORG = os.getenv("ORG")
GITHUB_ACCESS_TOKEN = os.getenv("GITHUB_ACCESS_TOKEN")
s3_endpoint_url = os.getenv("S3_ENDPOINT")
aws_access_key_id = os.getenv("S3_ACCESS_KEY")
aws_secret_access_key = os.getenv("S3_SECRET_KEY")
s3_bucket = os.getenv("S3_BUCKET")

In [3]:
# init s3 connector
s3c = S3Communication(
    s3_endpoint_url, aws_access_key_id, aws_secret_access_key, s3_bucket
)

In [4]:
repo_slug = f"{ORG}/{REPO}"
repo_slug

'os-climate/OS-Climate-Community-Hub'

In [5]:
# Note: The GitHub access token needs to be exported before importing the srcopmetrics package (current bug)
from srcopsmetrics.entities.issue import Issue  # noqa: E402
from srcopsmetrics.entities.pull_request import PullRequest  # noqa: E402

In [6]:
# Gather the data
!rm -r srcopsmetrics/
!python -m srcopsmetrics.cli -clr $repo_slug -e Issue,PullRequest

INFO:srcopsmetrics.github_knowledge:Overall repositories found: 1
INFO:srcopsmetrics.bot_knowledge:######################## Analysing os-climate/OS-Climate-Community-Hub ########################

INFO:srcopsmetrics.utils:No repo identified, creating new directory at /opt/app-root/src/metrics/notebooks/srcopsmetrics/bot_knowledge/os-climate/OS-Climate-Community-Hub
INFO:srcopsmetrics.bot_knowledge:########################
INFO:srcopsmetrics.bot_knowledge:Detected entities:
CodeFrequency # Commit # DependencyUpdate # Fork # Issue # IssueEvent # KebechetUpdateManager # License # PullRequest # PullRequestDiscussion # RawIssue # RawPullRequest # ReadMe # Release # Stargazer # TrafficClones # TrafficPaths # TrafficPaths # TrafficReferrers # TrafficClones # TrafficViews
INFO:srcopsmetrics.bot_knowledge:########################
INFO:srcopsmetrics.bot_knowledge:Issue inspection
INFO:srcopsmetrics.entities.tools.storage:Loading knowledge locally
INFO:srcopsmetrics.entities.tools.storage:Data fro

In [7]:
issue = Issue(repo_slug)
issue_df = issue.load_previous_knowledge(is_local=True)
issue_df.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions
id,,,,,,,,
35,Request Onboarding to OSC Data Commons,Requesting onboarding for github user: @romeok...,oindrillac,2022-05-23 18:05:16,None,NaT,"{'onboarding': {'color': 'DE643F', 'labeled_at...",{'HeatherAck': 13}
34,Request credentials for an os-climate bucket,Requesting credentials for github user: mriefe...,mriefer,2022-05-13 09:02:21,None,NaT,{},{}
33,Request credentials for an os-climate bucket,Requesting credentials for github user: Christ...,ChristianMeyndt,2022-05-13 08:59:09,None,NaT,{},{}
32,Request credentials for an os-climate bucket,Requesting credentials for github user: DaBeID...,DaBeIDS,2022-05-13 08:54:23,None,NaT,{},{}
31,Request Onboarding to OSC Data Commons,Requesting onboarding for github user: mandy-c...,mandy-chessell,2022-05-12 13:28:05,HeatherAck,2022-05-24 15:56:53,"{'onboarding': {'color': 'DE643F', 'labeled_at...",{'HeatherAck': 12}


In [8]:
issue_df = issue_df.reset_index()

In [9]:
issue_df.head()

,id,title,body,created_by,created_at,closed_by,closed_at,labels,interactions
0,35,Request Onboarding to OSC Data Commons,Requesting onboarding for github user: @romeok...,oindrillac,2022-05-23 18:05:16,None,NaT,"{'onboarding': {'color': 'DE643F', 'labeled_at...",{'HeatherAck': 13}
1,34,Request credentials for an os-climate bucket,Requesting credentials for github user: mriefe...,mriefer,2022-05-13 09:02:21,None,NaT,{},{}
2,33,Request credentials for an os-climate bucket,Requesting credentials for github user: Christ...,ChristianMeyndt,2022-05-13 08:59:09,None,NaT,{},{}
3,32,Request credentials for an os-climate bucket,Requesting credentials for github user: DaBeID...,DaBeIDS,2022-05-13 08:54:23,None,NaT,{},{}
4,31,Request Onboarding to OSC Data Commons,Requesting onboarding for github user: mandy-c...,mandy-chessell,2022-05-12 13:28:05,HeatherAck,2022-05-24 15:56:53,"{'onboarding': {'color': 'DE643F', 'labeled_at...",{'HeatherAck': 12}


In [10]:
# Retain only relevant columns
issue_cols_to_drop = ["labels", "interactions"]
issue_df = issue_df.drop(columns=issue_cols_to_drop)
issue_df["org"] = ORG
issue_df["repo"] = REPO

issue_df.head()

,id,title,body,created_by,created_at,closed_by,closed_at,org,repo
0,35,Request Onboarding to OSC Data Commons,Requesting onboarding for github user: @romeok...,oindrillac,2022-05-23 18:05:16,None,NaT,os-climate,OS-Climate-Community-Hub
1,34,Request credentials for an os-climate bucket,Requesting credentials for github user: mriefe...,mriefer,2022-05-13 09:02:21,None,NaT,os-climate,OS-Climate-Community-Hub
2,33,Request credentials for an os-climate bucket,Requesting credentials for github user: Christ...,ChristianMeyndt,2022-05-13 08:59:09,None,NaT,os-climate,OS-Climate-Community-Hub
3,32,Request credentials for an os-climate bucket,Requesting credentials for github user: DaBeID...,DaBeIDS,2022-05-13 08:54:23,None,NaT,os-climate,OS-Climate-Community-Hub
4,31,Request Onboarding to OSC Data Commons,Requesting onboarding for github user: mandy-c...,mandy-chessell,2022-05-12 13:28:05,HeatherAck,2022-05-24 15:56:53,os-climate,OS-Climate-Community-Hub


In [11]:
# Upload the processed df to s3 as a parquet file
s3c.upload_df_to_s3(
    df=issue_df,
    s3_prefix="open-services-group/metrics/github/os-climate/issues",
    s3_key=f"{ORG}-{REPO}.parquet",
)

{'ResponseMetadata': {'RequestId': 'tx000000000000000016cc3-00628fbd0a-f6835f-ocs-storagecluster-cephobjectstore',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '0',
   'etag': '"daa6440b3eb57b30a4584ca6147a1cdc"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx000000000000000016cc3-00628fbd0a-f6835f-ocs-storagecluster-cephobjectstore',
   'date': 'Thu, 26 May 2022 17:46:50 GMT',
   'set-cookie': 'bbdcd938787a45e68f8d240a4e2dadcf=9305a9992645bb0698c9f5d65ae10c7e; path=/; HttpOnly'},
  'RetryAttempts': 0},
 'ETag': '"daa6440b3eb57b30a4584ca6147a1cdc"'}

In [12]:
pr = PullRequest(repo_slug)
pr_df = pr.load_previous_knowledge(is_local=True)
pr_df.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at
id,,,,,,,,,,,,,,,,,,
29,Add OPF OSC board view,Signed-off-by: Erik Erlandson <eerlands@redhat...,XS,erikerlandson,2022-05-05 22:03:10,2022-05-05 22:03:43,erikerlandson,2022-05-05 22:03:43,erikerlandson,1,1,{},{},[],[18bbf9c1779aa1ad7474533a7d88e17f3a4ffc96],[README.md],NaT,NaT
21,Add request credentials issue template,Fixes #20 \r\n\r\n@erikerlandson,S,Shreyanand,2022-04-05 20:12:05,2022-04-06 16:06:15,erikerlandson,2022-04-06 16:06:15,erikerlandson,1,1,"{'Shreyanand': 1, 'erikerlandson': 4}",{},[],[7f495d13d687feb4cac247cd57f0ea8996ad1538],[.github/ISSUE_TEMPLATE/request-credentials-fo...,NaT,NaT
19,Onboarding template,Fixes #15,S,erikerlandson,2022-03-25 18:47:18,2022-03-28 14:58:29,HeatherAck,2022-03-28 14:58:29,HeatherAck,1,1,{'erikerlandson': 153},"{'923289967': {'author': 'HeatherAck', 'words_...",[],[fc18e481d703fc0039da25577a13ca545a38c273],[.github/ISSUE_TEMPLATE/request-onboarding-to-...,2022-03-28 14:57:07,2022-03-28 14:57:07
1,Thoth Configuration Initialization,## Automatic configuration initialization\nThe...,M,khebhut[bot],2022-01-28 18:51:07,2022-01-28 19:47:23,HeatherAck,2022-01-28 19:47:23,HeatherAck,1,1,{},"{'866646009': {'author': 'HeatherAck', 'words_...",[],[f2fe0ca0f227c78ca07bb883bbb493b52bcd3e35],[.thoth.yaml],2022-01-28 19:47:13,2022-01-28 19:47:13


In [13]:
pr_df = pr_df.reset_index()

In [14]:
pr_df.head()

,id,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at
0,29,Add OPF OSC board view,Signed-off-by: Erik Erlandson <eerlands@redhat...,XS,erikerlandson,2022-05-05 22:03:10,2022-05-05 22:03:43,erikerlandson,2022-05-05 22:03:43,erikerlandson,1,1,{},{},[],[18bbf9c1779aa1ad7474533a7d88e17f3a4ffc96],[README.md],NaT,NaT
1,21,Add request credentials issue template,Fixes #20 \r\n\r\n@erikerlandson,S,Shreyanand,2022-04-05 20:12:05,2022-04-06 16:06:15,erikerlandson,2022-04-06 16:06:15,erikerlandson,1,1,"{'Shreyanand': 1, 'erikerlandson': 4}",{},[],[7f495d13d687feb4cac247cd57f0ea8996ad1538],[.github/ISSUE_TEMPLATE/request-credentials-fo...,NaT,NaT
2,19,Onboarding template,Fixes #15,S,erikerlandson,2022-03-25 18:47:18,2022-03-28 14:58:29,HeatherAck,2022-03-28 14:58:29,HeatherAck,1,1,{'erikerlandson': 153},"{'923289967': {'author': 'HeatherAck', 'words_...",[],[fc18e481d703fc0039da25577a13ca545a38c273],[.github/ISSUE_TEMPLATE/request-onboarding-to-...,2022-03-28 14:57:07,2022-03-28 14:57:07
3,1,Thoth Configuration Initialization,## Automatic configuration initialization\nThe...,M,khebhut[bot],2022-01-28 18:51:07,2022-01-28 19:47:23,HeatherAck,2022-01-28 19:47:23,HeatherAck,1,1,{},"{'866646009': {'author': 'HeatherAck', 'words_...",[],[f2fe0ca0f227c78ca07bb883bbb493b52bcd3e35],[.thoth.yaml],2022-01-28 19:47:13,2022-01-28 19:47:13


In [15]:
# Retain only relevant columns
pr_cols_to_drop = ["interactions", "reviews", "labels", "commits", "changed_files"]
prs_df = pr_df.drop(columns=pr_cols_to_drop)
prs_df["org"] = ORG
prs_df["repo"] = REPO

prs_df.head()

,id,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,first_review_at,first_approve_at,org,repo
0,29,Add OPF OSC board view,Signed-off-by: Erik Erlandson <eerlands@redhat...,XS,erikerlandson,2022-05-05 22:03:10,2022-05-05 22:03:43,erikerlandson,2022-05-05 22:03:43,erikerlandson,1,1,NaT,NaT,os-climate,OS-Climate-Community-Hub
1,21,Add request credentials issue template,Fixes #20 \r\n\r\n@erikerlandson,S,Shreyanand,2022-04-05 20:12:05,2022-04-06 16:06:15,erikerlandson,2022-04-06 16:06:15,erikerlandson,1,1,NaT,NaT,os-climate,OS-Climate-Community-Hub
2,19,Onboarding template,Fixes #15,S,erikerlandson,2022-03-25 18:47:18,2022-03-28 14:58:29,HeatherAck,2022-03-28 14:58:29,HeatherAck,1,1,2022-03-28 14:57:07,2022-03-28 14:57:07,os-climate,OS-Climate-Community-Hub
3,1,Thoth Configuration Initialization,## Automatic configuration initialization\nThe...,M,khebhut[bot],2022-01-28 18:51:07,2022-01-28 19:47:23,HeatherAck,2022-01-28 19:47:23,HeatherAck,1,1,2022-01-28 19:47:13,2022-01-28 19:47:13,os-climate,OS-Climate-Community-Hub


In [16]:
# Upload the processed df to s3 as a parquet file
s3c.upload_df_to_s3(
    df=prs_df,
    s3_prefix="open-services-group/metrics/github/os-climate/prs",
    s3_key=f"{ORG}-{REPO}.parquet",
)

{'ResponseMetadata': {'RequestId': 'tx000000000000000016cc4-00628fbd0c-f6835f-ocs-storagecluster-cephobjectstore',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '0',
   'etag': '"2cb47de22b4b4ebc639d7c8ba34dc9e5"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx000000000000000016cc4-00628fbd0c-f6835f-ocs-storagecluster-cephobjectstore',
   'date': 'Thu, 26 May 2022 17:46:52 GMT',
   'set-cookie': 'bbdcd938787a45e68f8d240a4e2dadcf=9305a9992645bb0698c9f5d65ae10c7e; path=/; HttpOnly'},
  'RetryAttempts': 0},
 'ETag': '"2cb47de22b4b4ebc639d7c8ba34dc9e5"'}

In [17]:
# Map the datatype columns of df to suitable datatype supported in Trino/Superset
_p2smap = {
    "object": "varchar",
    "int64": "bigint",
    "float64": "double",
    "datetime64[ns]": "timestamp",
    "bool": "boolean",
}


def pandas_type_to_sql(pt):
    st = _p2smap.get(pt)
    if st is not None:
        return st
    raise ValueError("unexpected pandas column type '{pt}'".format(pt=pt))


# Generate the Trino table schema
def generate_table_schema_pairs(df):
    ptypes = [str(e) for e in df.dtypes.to_list()]
    stypes = [pandas_type_to_sql(e) for e in ptypes]
    pz = list(zip(df.columns.to_list(), stypes))
    return ",\n".join(["    {n} {t}".format(n=e[0], t=e[1]) for e in pz])

In [18]:
# Create a Trino client
conn = trino.dbapi.connect(
    auth=trino.auth.BasicAuthentication(
        os.environ["TRINO_USER"], os.environ["TRINO_PASSWD"]
    ),
    host=os.environ["TRINO_HOST"],
    port=int(os.environ["TRINO_PORT"]),
    http_scheme="https",
    verify=True,
)
cur = conn.cursor()

In [19]:
# Check if Trino connection was successful
cur.execute("show catalogs")
cur.fetchall()[1]

['data_science_general']

In [ ]:
# Create the issues table with data populated from parquet file
issue_schema = generate_table_schema_pairs(issue_df)

tabledef = """create table if not exists data_science_general.default.os_climate_issues(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{s3_bucket}/open-services-group/metrics/github/os-climate/issues'
)""".format(
    schema=issue_schema,
    s3_bucket=os.environ["S3_BUCKET"],
)

cur.execute(tabledef)
cur.fetchall()

In [ ]:
# Create the PR table with data populated from parquet file
pr_schema = generate_table_schema_pairs(prs_df)

tabledef = """create table if not exists data_science_general.default.os_climate_prs(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{s3_bucket}/open-services-group/metrics/github/os-climate/prs'
)""".format(
    schema=pr_schema,
    s3_bucket=os.environ["S3_BUCKET"],
)

cur.execute(tabledef)
cur.fetchall()